In [ ]:
import json
from vllm import LLM, SamplingParams
from tqdm import tqdm

print("库导入成功。")

In [ ]:
# --- 模型和数据路径配置 ---
MODEL_PATH = "/home/rmx/workspace/checkpoint-7700"  # 替换为你的模型路径或Hugging Face模型ID
DATA_PATH = "/home/rmx/workspace/aamas/test_data_1k.jsonl"             # 你的数据集文件路径
OUTPUT_PATH = "inference_results.jsonl"    # 结果保存路径

# --- 推理引擎配置 ---
TENSOR_PARALLEL_SIZE = 2  # 使用的GPU数量

# --- 采样参数配置 ---

TEMPERATURE = 0.2
TOP_P = 1.0
MAX_TOKENS = 2048 # 根据你任务的输出长度需求来调整

# 可以在这里设置停止词，防止模型生成多余内容, 比如 instruction block 的 "###"
STOP_TOKENS = ["###"]

print("参数配置完成。")
print(f"模型路径: {MODEL_PATH}")
print(f"数据路径: {DATA_PATH}")

In [ ]:
PROMPT_TEMPLATE = """
### Input:
{input}

### Response:
"""

print("提示模板已定义。")

In [ ]:
prompts = []
source_data = [] # 保存原始数据，以便后续合并结果

print(f"正在从 {DATA_PATH} 加载数据...")
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            data_item = json.loads(line.strip())
            
            # 如果 instruction 为空, 使用一个默认指令
            if not data_item.get("instruction"):
                data_item["instruction"] = ""

            # 美化 input 字段中的 JSON 字符串，使其更易读
            try:
                input_json = json.loads(data_item['input'])
                pretty_input = json.dumps(input_json, indent=4)
            except (json.JSONDecodeError, TypeError):
                pretty_input = data_item['input'] # 如果不是合法的JSON字符串，则直接使用

            # 构建完整的 prompt
            prompt = PROMPT_TEMPLATE.format(
                instruction=data_item['instruction'],
                input=pretty_input
            )
            prompts.append(prompt)
            source_data.append(data_item)
        except json.JSONDecodeError:
            print(f"跳过格式错误的行: {line.strip()}")

print(f"数据加载完成，共计 {len(prompts)} 条。")

# (可选) 打印第一条处理好的 prompt 进行检查
if prompts:
    print("\n--- 第一条 Prompt 示例 ---\n")
    print(prompts[0])

In [ ]:
print("正在加载模型到 vLLM...")
llm = LLM(model=MODEL_PATH, tensor_parallel_size=TENSOR_PARALLEL_SIZE, gpu_memory_utilization=0.8)
print("模型加载成功！")

# 定义采样参数
sampling_params = SamplingParams(
    temperature=TEMPERATURE,
    top_p=TOP_P,
    max_tokens=MAX_TOKENS,
    stop=STOP_TOKENS
)
print("采样参数设置完成。")

In [ ]:
print("开始批量推理...")
# vLLM 的 generate 方法会显示一个进度条
outputs = llm.generate(prompts, sampling_params)
print(f"批量推理完成，生成了 {len(outputs)} 条结果。")

In [ ]:
results = []
for i, output in enumerate(outputs):
    generated_text = output.outputs[0].text.strip()
    
    # 将原始数据和生成的结果合并
    result_item = source_data[i]
    result_item['generated_output'] = generated_text
    results.append(result_item)

# 写入到新的 jsonl 文件
print(f"正在将结果保存到 {OUTPUT_PATH}...")
with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
    for result in tqdm(results, desc="保存进度"):
        f.write(json.dumps(result, ensure_ascii=False) + '\n')

print("所有结果已成功保存！")